In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "1234",
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "from sklearn.impute import SimpleImputer\n",
    "from sklearn.preprocessing import LabelEncoder\n",
    "from sklearn.model_selection import train_test_split\n",
    "from tensorflow.keras.models import Sequential\n",
    "from tensorflow.keras.layers import Dense\n",
    "import matplotlib.pyplot as plt\n",
    "import streamlit as st\n",
    "\n",
    "# โหลดข้อมูล DDoS\n",
    "file_path = '/mnt/data/DDoS_Dataset_with_Missing_Values.csv'\n",
    "ddos_data = pd.read_csv(file_path)\n",
    "\n",
    "# การจัดการค่าที่หายไปในข้อมูลการโจมตี DDoS\n",
    "columns_to_consider = ddos_data.drop(columns=['Highest Layer', 'Transport Layer', 'Source IP', 'Dest IP'])\n",
    "\n",
    "# เติมค่าที่หายไปในข้อมูลเชิงตัวเลข\n",
    "imputer = SimpleImputer(strategy='mean')  # เติมค่าที่หายไปด้วยค่าเฉลี่ย\n",
    "columns_to_consider[['Source Port', 'Dest Port', 'Packet Length', 'Packets/Time']] = imputer.fit_transform(\n",
    "    columns_to_consider[['Source Port', 'Dest Port', 'Packet Length', 'Packets/Time']]\n",
    ")\n",
    "\n",
    "# แปลงข้อมูลประเภทข้อความใน target เป็นตัวเลข\n",
    "label_encoder = LabelEncoder()\n",
    "columns_to_consider['target'] = label_encoder.fit_transform(columns_to_consider['target'])\n",
    "\n",
    "# แสดงผลข้อมูลที่เติมค่าหายไปแล้ว\n",
    "columns_to_consider.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "5678",
   "metadata": {},
   "outputs": [],
   "source": [
    "# แยกข้อมูลเป็น X (features) และ y (target)\n",
    "X = columns_to_consider.drop(columns=['target'])\n",
    "y = columns_to_consider['target']\n",
    "\n",
    "# แบ่งข้อมูลเป็นชุดฝึก (Training Set) และชุดทดสอบ (Test Set)\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n",
    "\n",
    "# สร้าง Neural Network Model\n",
    "model = Sequential()\n",
    "model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # ชั้นแรก 64 นิวรอน\n",
    "model.add(Dense(32, activation='relu'))  # ชั้นที่สอง 32 นิวรอน\n",
    "model.add(Dense(1, activation='sigmoid'))  # เลเยอร์สุดท้ายสำหรับ Classification (2 คลาส)\n",
    "\n",
    "# คอมไพล์โมเดล\n",
    "model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])\n",
    "\n",
    "# ฝึกโมเดล\n",
    "history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))\n",
    "\n",
    "# แสดงผลลัพธ์การฝึก\n",
    "train_accuracy = history.history['accuracy'][-1]\n",
    "test_accuracy = history.history['val_accuracy'][-1]\n",
    "train_accuracy, test_accuracy"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "9101",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Train Accuracy: 0.9340\n",
       "Test Accuracy: 0.9123"
      ]
     },
     "execution_count": 3,
     "metadata": {},
     "output_type": "stream"
    }
   ],
   "source": [
    "# แสดงผลลัพธ์การฝึก\n",
    "print(f'Train Accuracy: {train_accuracy:.4f}')\n",
    "print(f'Test Accuracy: {test_accuracy:.4f}')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "1122",
   "metadata": {},
   "outputs": [],
   "source": [
    "# แสดงกราฟการฝึก\n",
    "plt.plot(history.history['accuracy'], label='Train Accuracy')\n",
    "plt.plot(history.history['val_accuracy'], label='Test Accuracy')\n",
    "plt.title('Model Accuracy')\n",
    "plt.xlabel('Epochs')\n",
    "plt.ylabel('Accuracy')\n",
    "plt.legend()\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
